In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
from sklearn.metrics import accuracy_score
from concurrent.futures import ThreadPoolExecutor

import time

start_time = time.time()

IMG_SIZE = 50
DATASET_PATH = 'C:/Users/Hp/Downloads/dogs-vs-cats/train'


In [2]:
def process_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img.flatten()

def load_images_parallel():
    X = []
    y = []
    with ThreadPoolExecutor() as executor:
        for label in os.listdir(DATASET_PATH):
            for file in os.listdir(os.path.join(DATASET_PATH, label)):
                img_path = os.path.join(DATASET_PATH, label, file)
                X.append(executor.submit(process_image, img_path))
                y.append(label)
    X = np.array([future.result() for future in X])
    return X, np.array(y)

In [3]:
X, y = load_images_parallel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)

In [ ]:
joblib.dump(svm, 'C:/Users/Hp/Downloads/Train model/model.joblib')

y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

end_time = time.time()
execution_time = end_time - start_time
print("Execution Time:", execution_time, "seconds")